In [1]:
import os
import json
import openai
import requests
from dotenv import load_dotenv
from tenacity import retry, wait_random_exponential, stop_after_attempt


GPT_MODEL = "gpt-4-0613"

load_dotenv(".env")
openai.api_key = os.environ['OPENAI_API_KEY']

@retry(wait=wait_random_exponential(min=1, max=40), stop=stop_after_attempt(3))
def chat_completion_request(messages, functions=None, model=GPT_MODEL):
    headers = {
        "Content-Type": "application/json",
        "Authorization": "Bearer " + openai.api_key,
    }
    json_data = {"model": model, "messages": messages}
    if functions is not None:
        json_data.update({"functions": functions})
    try:
        response = requests.post(
            "https://api.openai.com/v1/chat/completions",
            headers=headers,
            json=json_data,
        )
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e
    
class Chat:
    def __init__(self):
        self.conversation_history = []

    def add_prompt(self, role: str, content: str):
        message = {"role": role, "content": content}
        self.conversation_history.append(message)
        
    def add_assistant_prompt(self, content: str):
        role = "assistant"
        self.add_prompt(role, content)
        
    def add_user_prompt(self, content: str):
        role = "user"
        self.add_prompt(role, content)

    def display_conversation(self):
        for message in self.conversation_history:
            print(
                f"{message['role']}: {message['content']}\n\n",
                message["role"],
            )
    def upload_conversation_history(conversation_history: list)
        self.conversation_history = conversation_history


In [2]:
SYSTEM_SETUP_PROMPT = \
    """
    You are a polite and smart AI assistant that helps people to fill questionire to apply for an insurance.
    We need to fill next questions:
    1) What is your first name?
    2) What is your last name?
    3) What is the type of insurance you need?
    4) What is your phone number?
    5) What is your age?

    We expect final response in json format with keys: "first_name", "last_name", "age", "type_of_insurance", "phone_number". 

    Allowed types of insurance are: "Auto", "Home", "Condo", "Tenant", "Farm", "Commercial", "Life".
    
    Make sure that the phone number either has 10 digits or (11 digits and starts with +1). 
    Don't save +1 for the phone number, we need only next 10 digits. Store as int.
    
    Age should be int value with year granularity, don't accept a string.    
    
    Please ask one question at a time.
    """

In [3]:
FUNCTIONS = [
    {
        "name": "save_users_questionire",
        "description": "If user responded all questiones, store fully filled questionire to the database",
        "parameters": {
            "type": "object",
            "properties": {
                "user_answers": {
                    "type": "object",
                    "description": "Keys of the dict are questions to the user and values are user's responses to the coresponding questions",
                },
            },
            "required": ["user_answers"],
        },
    },
    {
        "name": "ask_follow_up_question",
        "description": "If the user didn't answer all the questions, generates an additional question to ask user.",
        "parameters": {
            "type": "object",
            "properties": {
                "next_question": {
                    "type": "string",
                    "description": "Next question which we will ask user to clarify their response",
                },
            },
            "required": ["next_question"],
        },
    },
]

In [4]:
def generate_response(chat: object, functions: list = FUNCTIONS) -> (bool, str):
    
    chat_response = chat_completion_request(
        chat.conversation_history,
        functions=functions
    )
    
    if chat_response is not None:
        response_content = chat_response.json()['choices'][0]['message']

        message = chat_response.json()['choices'][0]['message']['content']

#         print("message:") 
#         print(message)

        if message is not None:
            
            chat_finished = False
            return (chat_finished, message)


        if 'function_call' in response_content:
            if response_content['function_call']['name'] == 'save_users_questionire':

                questionire = json.loads(response_content['function_call']['arguments'])
#                 print("Result questionire:")
#                 print(questionire)

                chat_finished = True
                return (chat_finished, questionire)

            elif response_content['function_call']['name'] == 'ask_follow_up_question':
                next_question = json.loads(response_content['function_call']['arguments'])['next_question']
#                 print("Next question:")
#                 print(next_question)

                chat_finished = False
                return (chat_finished, next_question)           

    else:
        print("ChatCompletion request failed. Retrying...")

### Chat

In [5]:
chat = Chat()
chat.add_prompt("system", SYSTEM_SETUP_PROMPT)

user_first_message = "Hi, I'm Bob Smith. I'm looking for a car insurance. Do you offer it?"
chat.add_user_prompt(user_first_message)

In [6]:
chat_finished, message = generate_response(chat)

chat.add_assistant_prompt(message)

chat_finished, message

(False,
 'Yes, we do offer car insurance. To complete your application, I would need some additional information. Could you please provide your phone number?')

In [7]:
user_message = "+19876543210"
chat.add_prompt("user", user_message)

In [8]:
chat_finished, message = generate_response(chat)

chat.add_assistant_prompt(message)

chat_finished, message

(False, 'What is your age?')

In [9]:
user_message = "24"
chat.add_prompt("user", user_message)

chat_finished, message = generate_response(chat)

chat_finished, message

(True,
 {'first_name': 'Bob',
  'last_name': 'Smith',
  'age': 24,
  'type_of_insurance': 'Auto',
  'phone_number': 9876543210})

In [10]:
chat.conversation_history

[{'role': 'system',
  'content': '\n    You are a polite and smart AI assistant that helps people to fill questionire to apply for an insurance.\n    We need to fill next questions:\n    1) What is your first name?\n    2) What is your last name?\n    3) What is the type of insurance you need?\n    4) What is your phone number?\n    5) What is your age?\n\n    We expect final response in json format with keys: "first_name", "last_name", "age", "type_of_insurance", "phone_number". \n\n    Allowed types of insurance are: "Auto", "Home", "Condo", "Tenant", "Farm", "Commercial", "Life".\n    \n    Make sure that the phone number either has 10 digits or (11 digits and starts with +1). \n    Don\'t save +1 for the phone number, we need only next 10 digits. Store as int.\n    \n    Age should be int value with year granularity, don\'t accept a string.    \n    \n    Please ask one question at a time.\n    '},
 {'role': 'user',
  'content': "Hi, I'm Bob Smith. I'm looking for a car insurance. 